<a href="https://colab.research.google.com/github/NbtKmy/gc_workshops/blob/main/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vektorisierung oder Embedding

Die Vektorisierung bei NLP bedeutet, dass man die Elemente in einem Textkorpus wie Wörter, Sätze und Texte in Vektor-Werte umwandelt. Durch Vektorisierung können die Ähnlichkeit oder Distanz zwischen Wörter, Sätze oder Texte "berechnet" werden.

Vektorisierung von Wörter, Sätze oder Texte werden auch Word-, Sentence- oder Text-Embedding bezeichnet.

In diesem Beispiel probieren wir die Word-Embedding.
Für dieses Verfahren verwenden wir die Library "[gensim](https://radimrehurek.com/gensim/)".

Als Beispiel nehmen wir den Roman "Frankenstein" von Shelley.

Der ganze Prozess verläuft so:

1. Preprozess - Der gesamte Text wird in 2-dimensionalen Array umgewandelt. Dabei wird Satz-Einheit berücksichtigt. Darüber hinaus werden nur bestimmte POS für die Vektorisierung herausgenommen.

1. Vektorisierung mit Word2Vec.

In [ ]:
!pip install -q spacy gensim spacy[transformers]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.7 MB/s eta 0:00:00


In [ ]:
from spacy.cli import download
download("en_core_web_trf")


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


## Preprozess


Um den ganzen Roman zu bearbeiten, dauert es ca. 11 min...
Bei der Tokenisierung wurde hier alle Wörter lemmatisiert, obwohl einige Entitäten wie "St. Petersburt" usw. aus mehr als ein Wort bestehen.
Hier wurde solche Fälle einfachhalber so belassen. Aber bei der Forschung wäre wichtig, solche Entitäten zu berücksichtigen.

In [ ]:
%%time
from spacy.lang.en import English
import spacy
import spacy_transformers

nlp_sent = English()
nlp_sent.add_pipe("sentencizer")

# Text-Daten einlesen
with open("frankenstein_full.txt") as f:
	  text = f.read()


doc = nlp_sent(text)


positiv_index = ["ADJ", "ADV", "NOUN", "PROPN", "VERB", "X"]

nlp = spacy.load("en_core_web_trf")

full_arr = []
for s in doc.sents:
    s = str(s)
    sent_analyse = nlp(s)
    sent_arr = []
    for token in sent_analyse:
        if token.pos_ in positiv_index:
            sent_arr.append(token.lemma_)
    if len(sent_arr) != 0:
        full_arr.append(sent_arr)

print(full_arr)

[['letter', 'Mrs.', 'Seville', 'England', 'St.', 'Petersburgh', 'Dec.', '11th', 'rejoice', 'hear', 'disaster', 'accompany', 'commencement', 'enterprise', 'regard', 'such', 'evil', 'foreboding'], ['arrive', 'here', 'yesterday', 'first', 'task', 'assure', 'dear', 'sister', 'welfare', 'increase', 'confidence', 'success', 'undertaking'], ['already', 'far', 'north', 'London', 'walk', 'street', 'Petersburgh', 'feel', 'cold', 'northern', 'breeze', 'play', 'cheek', 'brace', 'nerve', 'fill', 'delight'], ['understand', 'feeling'], ['breeze', 'travel', 'region', 'advance', 'give', 'foretaste', 'icy', 'clime'], ['inspirit', 'wind', 'promise', 'daydream', 'become', 'more', 'fervent', 'vivid'], ['try', 'vain', 'persuade', 'pole', 'seat', 'frost', 'desolation', 'ever', 'present', 'imagination', 'region', 'beauty', 'delight'], ['there', 'Margaret', 'sun', 'forever', 'visible', 'broad', 'disk', 'just', 'skirt', 'horizon', 'diffuse', 'perpetual', 'splendour'], ['there', 'leave', 'sister', 'put', 'trust'

In [ ]:
from gensim.models import word2vec

model = word2vec.Word2Vec(
            full_arr,
            vector_size=300,
            window=5,
            min_count=3,
            sg=1)

model.save("./w2v.model")
model.wv.save_word2vec_format("./frankenstein_w2v.pt", binary=True)

In [ ]:
# Wenn man das fertige Model verwenden will
# Zuerst die "model"-File hochladen

import gensim

# Load pre-trained Word2Vec model.
model = gensim.models.Word2Vec.load("frankenstein_w2v.model")
print(model.wv.most_similar("Frankenstein"))

[('tell', 0.9994692802429199), ('point', 0.9994688630104065), ('instant', 0.9994654655456543), ('afterwards', 0.9994595050811768), ('victim', 0.9994591474533081), ('wander', 0.9994535446166992), ('reflect', 0.9994466304779053), ('assure', 0.9994461536407471), ('sister', 0.9994456768035889), ('prepare', 0.9994454383850098)]


In [ ]:
print(model.wv.most_similar("Frankenstein"))

[('tell', 0.9995097517967224), ('point', 0.9994792938232422), ('afterwards', 0.9994608759880066), ('instant', 0.9994537234306335), ('lose', 0.9994518160820007), ('wander', 0.9994422793388367), ('victim', 0.9994419813156128), ('try', 0.9994398951530457), ('sister', 0.999439537525177), ('reflect', 0.9994391202926636)]


In [ ]:
# scipy und Plotly sind bereits installiert...
!pip install -qU umap-learn scipy plotly


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Visualisierung

ich bedanke mich bei dem Verfasser "@hima2b4" für [diesen Artikel](https://qiita.com/hima2b4/items/23a1dfad16321c7f23ae)!



In [ ]:
#@title UMAP
#@markdown  **<font color= "Crimson">Schritt 1</font>：Je nach dem Wert von "n_neighbors" ändert sich Clustering**

#@markdown  **<font color= "Crimson">Schritt 2</font>：Durch max_words_ratio kann man die Wörterzahl kontrollieren**

n_neighbors = 16 #@param {type:"slider", min:2, max:100, step:1}
UMAP_max_words_ratio = 0.2 #@param {type:"slider", min:0.1, max:1, step:0.05}
UMAP_text_font_size = 10 #@param {type:"slider", min:8, max:24, step:1}

import numpy as np
import pandas as pd
import umap.umap_ as umap
from scipy.sparse.csgraph import connected_components
import plotly.graph_objs as go
import plotly.express as px
import os

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['PYTHONHASHSEED'] = '0'

UMAP_max_words = int(len(model.wv.vectors)*UMAP_max_words_ratio)

np.set_printoptions(suppress=True)
values1 = umap.UMAP(n_components=2,n_neighbors=n_neighbors).fit_transform(model.wv.vectors)


fig_title ='<b>UMAP on word2vec embeddings'

# Visualisierung
fig1 = go.Figure()
for value, word in zip(values1, model.wv.index_to_key[0:UMAP_max_words]):
  fig1.add_trace(
      go.Scatter(
          x = pd.Series(value[0]),
          y = pd.Series(value[1]),
          mode = 'markers+text',
          text = word,
          textposition="top center"
          )
)
fig1.update_layout(title=dict(text = fig_title,
                             font=dict(size=18,
                                       color='grey'),
                             xref='paper', # container or paper
                             x=0.5,
                             y=0.9,
                             xanchor='center'
                            ),
                  showlegend=False,
                  font = dict(size = 10),
                  width=900,
                  height=700
                   )
fig1.show()

